# To install and run Jupyter Notebook
```
$ apt install virtualenv
$ virtualenv -p python2 env
$ source env/bin/activate
$ pip install pwntools jupyter r2pipe
$ export PWNLIB_NOTERM=true
$ jupyter notebook
```

# Creating an executable from assembly code

In [21]:
from pwn import *
context.arch = "amd64"
context.bits = 64

s = "".join([chr(ord(i)-10) for i in "flag{letsgetbusy}"])
ss = [u64(s[i:i+8].ljust(8, '\x00')) for i in range(0, len(s), 8)]
print("The pieces: {}, len: {}".format([hex(i) for i in ss], len(ss)))

The pieces: ['0x6a5b62715d57625c', '0x6f696b586a5b5d69', '0x73'], len: 3


In [38]:
from pwn import *
context.arch = "amd64"
context.bits = 64

asmcode = """
MAIN:
    mov rax, {}
    push rax
    mov rax, {}
    push rax
    mov rax, {}
    push rax
    mov rdx, rsp
    mov rcx, 0
    
CHECK:
    cmp rcx, {}
    je END
    
    mov al, BYTE PTR[rdx]
    add al, 10
    mov BYTE PTR[rdx], al
    inc rdx
    inc rcx
    jmp CHECK

END:
    mov rax, 60 # exit syscall
    syscall
    ret
""".format(ss[2], ss[1], ss[0], len(s))

print(disasm(asm(asmcode)))
with open("runme", "wb") as f:
    f.write(make_elf(asm(asmcode)))

DEBUG:pwnlib.asm:cpp -C -nostdinc -undef -P -I/home/erfur/Downloads/ctf/env/lib/python2.7/site-packages/pwnlib/data/includes /dev/stdin
DEBUG:pwnlib.asm:Assembling
.section .shellcode,"awx"
.global _start
.global __start
_start:
__start:
.intel_syntax noprefix
MAIN:
    mov rax, 115
    push rax
    mov rax, 8028065838250286441
    push rax
    mov rax, 7663827429937865308
    push rax
    mov rdx, rsp
    mov rcx, 0
CHECK:
    cmp rcx, 17
    je END
    mov al, BYTE PTR[rdx]
    add al, 10
    mov BYTE PTR[rdx], al
    inc rdx
    inc rcx
    jmp CHECK
END:
    mov rax, 60 # exit syscall
    syscall
    ret

DEBUG:pwnlib.asm:/usr/bin/as -64 -o /tmp/pwn-asm-qs8Dp3/step2 /tmp/pwn-asm-qs8Dp3/step1
DEBUG:pwnlib.asm:/usr/bin/objcopy -j .shellcode -Obinary /tmp/pwn-asm-qs8Dp3/step3 /tmp/pwn-asm-qs8Dp3/step4
DEBUG:pwnlib.asm:/usr/bin/objcopy -I binary -O elf64-x86-64 -B i386:x86-64 --set-section-flags .data=code --rename-section .data=.text -w -N * /tmp/pwn-disasm-DUs0Cp/step1 /tmp/pwn-disas

   0:   48 c7 c0 73 00 00 00    mov    rax,0x73
   7:   50                      push   rax
   8:   48 b8 69 5d 5b 6a 58    movabs rax,0x6f696b586a5b5d69
   f:   6b 69 6f 
  12:   50                      push   rax
  13:   48 b8 5c 62 57 5d 71    movabs rax,0x6a5b62715d57625c
  1a:   62 5b 6a 
  1d:   50                      push   rax
  1e:   48 89 e2                mov    rdx,rsp
  21:   48 c7 c1 00 00 00 00    mov    rcx,0x0
  28:   48 83 f9 11             cmp    rcx,0x11
  2c:   74 0e                   je     0x3c
  2e:   8a 02                   mov    al,BYTE PTR [rdx]
  30:   04 0a                   add    al,0xa
  32:   88 02                   mov    BYTE PTR [rdx],al
  34:   48 ff c2                inc    rdx
  37:   48 ff c1                inc    rcx
  3a:   eb ec                   jmp    0x28
  3c:   48 c7 c0 3c 00 00 00    mov    rax,0x3c
  43:   0f 05                   syscall 
  45:   c3                      ret


# Analyzing an executable with Radare2
_Requires Radare2 to be installed_

In [36]:
import r2pipe
import json

r2 = r2pipe.open("runme")
r2.cmd("e asm.comments=false")
r2.cmd("e scr.utf8=false")
# aa: basic analysis
r2.cmd("aa")

# iej: entrypoint(json output)
entry = json.loads(r2.cmd("iej"))[0]["vaddr"]
print("Entrypoint: 0x{:0x}".format(entry))

# aflq: function list(quiet output)
for f in r2.cmd("aflq").split():
    # pdf: disassemble function
    print(r2.cmd("pdf @{}".format(f)))

r2.quit()

Entrypoint: 0x401000
            ;-- section..shellcode:
            ;-- segment.LOAD0:
            ;-- segment.ehdr:
            ;-- rip:
/ (fcn) entry0 70
|   entry0 ();
|           0x00401000      48c7c0730000.  mov rax, 0x73
|           0x00401007      50             push rax
|           0x00401008      48b8695d5b6a.  movabs rax, 0x6f696b586a5b5d69
|           0x00401012      50             push rax
|           0x00401013      48b85c62575d.  movabs rax, 0x6a5b62715d57625c
|           0x0040101d      50             push rax
|           0x0040101e      4889e2         mov rdx, rsp
|           0x00401021      48c7c1000000.  mov rcx, 0
|       .-> 0x00401028      4883f911       cmp rcx, 0x11
|      ,==< 0x0040102c      740e           je 0x40103c
|      |:   0x0040102e      8a02           mov al, byte [rdx]
|      |:   0x00401030      040a           add al, 0xa
|      |:   0x00401032      8802           mov byte [rdx], al
|      |:   0x00401034      48ffc2         inc rdx
|      |:   0x0

# Concepts we've gone through

* CPU - Memory
* Assembly
* Machine Code
* Register
* Stack frame
* Calling convention
* Disassembler
* Debugger

# Reference Links

### x86 assembly
- http://www.cs.virginia.edu/~evans/cs216/guides/x86.html
- https://www.felixcloutier.com/x86/

### Stack Frame
- https://en.wikipedia.org/wiki/Call_stack

### Calling Convention
- https://en.wikipedia.org/wiki/Calling_convention
- https://en.wikipedia.org/wiki/X86_calling_conventions

### Disassembler
- https://www.hex-rays.com/products/ida/support/download_freeware.shtml
- https://ghidra-sre.org/
- https://rada.re/r/

### Debugger
- https://github.com/longld/peda

# GDB(GNU Debugger) cheatsheet

```$ gdb runme```

### Breakpoints

Using an address (eg: 0x40080):

```$ break *0x40080```

Using a function name:

```$ break main```

### Running and stepping through instructions

Run the executable:

```$ run```

Step through instructions (without going into the function):

```$ nexti```

Step through instructions (with going into the function):

```$ stepi```

Note: Hitting ```enter``` will execute the last command.

### Analyzing a function

Using peda:

```$ pdisass main```